<a href="https://colab.research.google.com/github/fjadidi2001/DataScienceJourney/blob/master/DANES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install selenium


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 29.3 MB/s eta 0:00:00


In [10]:
import requests
from bs4 import BeautifulSoup
import re
import facebook_scraper as fs

def get_facebook_post_text(url):
    """
    Extract text content from a Facebook post URL.

    Args:
        url (str): Facebook post URL

    Returns:
        str: Text content of the post
    """
    try:
        # Extract post ID and page ID from the URL
        post_id_match = re.search(r'posts/(\d+)', url)

        if not post_id_match:
            return "Could not extract post ID from URL"

        post_id = post_id_match.group(1)

        # Use facebook-scraper library to get post content
        # Note: Facebook's anti-scraping measures may require authentication
        posts = list(fs.get_posts(post_ids=[post_id], pages=1))

        if not posts:
            return "No post found with this ID"

        return posts[0]['text']

    except Exception as e:
        return f"Error retrieving post content: {str(e)}"

# Example usage
url = "https://www.facebook.com/ABCNewsPolitics/posts/1035057923259100"
news_text = get_facebook_post_text(url)
print(news_text)

ModuleNotFoundError: No module named 'facebook_scraper'